In [1]:

import StockPredictorModule as spm
import DecisionTree as dt
import schedule, time
#Imports
from pyrh import Robinhood
import dotenv
dotenv.load_dotenv()
import os
import importlib.util
import numpy as np
    
def RHlogin():
    import os
    USERNAME = os.getenv("RH_USERNAME")
    PASSWORD = os.getenv("RH_PASSWORD")

    rh = Robinhood(username=USERNAME, password=PASSWORD)
    rh.login()
    return rh

    
def inspect_historicals(rh, stock:str):
    print("Getting Historical Quotes")
     # Get 5 minute graph data for Ford stock
    historical_quotes = rh.get_historical_quotes("TWTR", "5minute", "day")
    print(historical_quotes['results'][0]["historicals"])


def plot_future_predictions(stock:str):
    stock_predict = spm.Stock_Predict(stock)
    stock_predict.plot_future_predictions()
    

In [ ]:
if __name__ == "__main__":
  
    ###GET CSV FILES TO TRAIN DECISION TREE###
    csvdata = dt.CSV_Maker()
    stocklist = ["NFLX", "AAPL", "BAC", "GE", "F"]    
    csvdata.add_csv_data(stocklist)                      
    csvdata.read_csv()
    csvdata.encode_csv()                                 
    
    ### IDENTIFYING THE SUCCESS AND FAILURE VARIABLES ###
    
    #s = sched.scheduler(time.time, time.sleep)
    
    
    
    def should_buy(stock:str):
        tree = dt.StockDecisionTree()
        buy = tree.make_prediction(csvdata.return_row(stock))
        return buy
    
    def visualize_predictions(stocklist:[]):
        for i in range(len(stocklist)):
            plot_future_predictions(stocklist[i])
    
    visualize_predictions(stocklist)
    
    def get_instrument(stock,rh):
        instrument = rh.instruments(stock)
        return instrument
    
    
    def run_script():
        import numpy as np
        success = np.array([1])
        fail = np.array([0])
        rh = RHlogin()
        #inspect_historicals(rh)
        instrument = get_instrument("F",rh)
        buy = should_buy("F")
        traded = False
    
        if (buy ==  np.array([1])):
            rh.place_buy_order(instrument, 1)
            traded = True
    
        if (buy == np.array([0]) and traded == True):
            rh.place_sell_order(instrument,1)
    

    #s.enter(300, 1, run_script, (s,))
    #s.run()
    schedule.every(300).seconds.do(run_script)
    
    while True:
        schedule.run_pending()
        time.sleep(1)
       
   

    

Epoch 1/50
6/6 [==============================] - 10s 747ms/step - loss: 0.0228 - val_loss: 0.0060
Epoch 2/50
6/6 [==============================] - 1s 193ms/step - loss: 0.0058 - val_loss: 0.0851
Epoch 3/50
6/6 [==============================] - 1s 218ms/step - loss: 0.0044 - val_loss: 0.0364
Epoch 4/50
6/6 [==============================] - 1s 220ms/step - loss: 0.0023 - val_loss: 0.0056
Epoch 5/50
6/6 [==============================] - 1s 162ms/step - loss: 0.0015 - val_loss: 0.0142
Epoch 6/50
6/6 [==============================] - 1s 164ms/step - loss: 0.0013 - val_loss: 0.0042
Epoch 7/50
6/6 [==============================] - 1s 161ms/step - loss: 8.2052e-04 - val_loss: 0.0106
Epoch 8/50
6/6 [==============================] - 1s 160ms/step - loss: 6.2498e-04 - val_loss: 0.0118
Epoch 9/50
6/6 [==============================] - 1s 161ms/step - loss: 5.5873e-04 - val_loss: 0.0230
Epoch 10/50
6/6 [==============================] - 1s 163ms/step - loss: 5.6730e-04 - val_loss: 0.0100
Ep

6/6 [==============================] - 1s 168ms/step - loss: 1.5811e-04 - val_loss: 0.0091
Epoch 24/50
6/6 [==============================] - 1s 171ms/step - loss: 1.6456e-04 - val_loss: 0.0090
Epoch 25/50
6/6 [==============================] - 1s 169ms/step - loss: 1.5938e-04 - val_loss: 0.0091
Epoch 26/50
6/6 [==============================] - 1s 174ms/step - loss: 1.7120e-04 - val_loss: 0.0089
Epoch 27/50
6/6 [==============================] - 1s 169ms/step - loss: 1.5037e-04 - val_loss: 0.0089
Epoch 28/50
6/6 [==============================] - 1s 172ms/step - loss: 1.7164e-04 - val_loss: 0.0087
Epoch 29/50
6/6 [==============================] - 1s 169ms/step - loss: 1.5699e-04 - val_loss: 0.0088
Epoch 30/50
6/6 [==============================] - 1s 171ms/step - loss: 1.6860e-04 - val_loss: 0.0089
Epoch 31/50
6/6 [==============================] - 1s 170ms/step - loss: 1.8825e-04 - val_loss: 0.0086
Epoch 32/50
6/6 [==============================] - 1s 172ms/step - loss: 1.5896e-04 -

6/6 [==============================] - 1s 164ms/step - loss: 7.6097e-04 - val_loss: 0.0028
Epoch 46/50
6/6 [==============================] - 1s 162ms/step - loss: 6.8806e-04 - val_loss: 0.0031
Epoch 47/50
6/6 [==============================] - 1s 164ms/step - loss: 7.4683e-04 - val_loss: 0.0030
Epoch 48/50
6/6 [==============================] - 1s 157ms/step - loss: 8.1527e-04 - val_loss: 0.0028
Epoch 49/50
6/6 [==============================] - 1s 159ms/step - loss: 6.6246e-04 - val_loss: 0.0027
Epoch 50/50
6/6 [==============================] - 1s 161ms/step - loss: 7.6399e-04 - val_loss: 0.0030
1 day output [[1.0603184]]
2 day output [[1.0630761]]
3 day output [[1.0674524]]
4 day output [[1.0733789]]
5 day output [[1.0806648]]
6 day output [[1.0890754]]
7 day output [[1.098381]]
8 day output [[1.1083807]]
9 day output [[1.1189129]]
10 day output [[1.1298566]]
11 day output [[1.1411258]]
12 day output [[1.152664]]
13 day output [[1.1644356]]
14 day output [[1.1764205]]
15 day output

Epoch 10/50
6/6 [==============================] - 1s 167ms/step - loss: 0.0023 - val_loss: 0.0175
Epoch 11/50
6/6 [==============================] - 1s 160ms/step - loss: 0.0021 - val_loss: 0.0174
Epoch 12/50
6/6 [==============================] - 1s 163ms/step - loss: 0.0021 - val_loss: 0.0154
Epoch 13/50
6/6 [==============================] - 1s 164ms/step - loss: 0.0019 - val_loss: 0.0166
Epoch 14/50
6/6 [==============================] - 1s 163ms/step - loss: 0.0020 - val_loss: 0.0159
Epoch 15/50
6/6 [==============================] - 1s 163ms/step - loss: 0.0020 - val_loss: 0.0154
Epoch 16/50
6/6 [==============================] - 1s 174ms/step - loss: 0.0020 - val_loss: 0.0153
Epoch 17/50
6/6 [==============================] - 1s 190ms/step - loss: 0.0019 - val_loss: 0.0152
Epoch 18/50
6/6 [==============================] - 1s 199ms/step - loss: 0.0020 - val_loss: 0.0150
Epoch 19/50
6/6 [==============================] - 1s 182ms/step - loss: 0.0019 - val_loss: 0.0144
Epoch 20/5

6/6 [==============================] - 1s 178ms/step - loss: 4.8407e-04 - val_loss: 0.0410
Epoch 30/50
6/6 [==============================] - 1s 185ms/step - loss: 4.3798e-04 - val_loss: 0.0342
Epoch 31/50
6/6 [==============================] - 1s 178ms/step - loss: 4.3727e-04 - val_loss: 0.0331
Epoch 32/50
6/6 [==============================] - 1s 176ms/step - loss: 3.9691e-04 - val_loss: 0.0424
Epoch 33/50
6/6 [==============================] - 1s 188ms/step - loss: 4.5792e-04 - val_loss: 0.0274
Epoch 34/50
6/6 [==============================] - 1s 184ms/step - loss: 5.0458e-04 - val_loss: 0.0346
Epoch 35/50
6/6 [==============================] - 1s 194ms/step - loss: 4.4883e-04 - val_loss: 0.0475
Epoch 36/50
6/6 [==============================] - 1s 196ms/step - loss: 4.3537e-04 - val_loss: 0.0291
Epoch 37/50
6/6 [==============================] - 1s 184ms/step - loss: 3.6549e-04 - val_loss: 0.0466
Epoch 38/50
6/6 [==============================] - 1s 179ms/step - loss: 4.6568e-04 -

...

Epoch 18/50
6/6 [==============================] - 1s 141ms/step - loss: 0.0010 - val_loss: 0.0017
Epoch 19/50
6/6 [==============================] - 1s 137ms/step - loss: 9.6792e-04 - val_loss: 0.0018
Epoch 20/50
6/6 [==============================] - 1s 133ms/step - loss: 9.9908e-04 - val_loss: 0.0019
Epoch 21/50
6/6 [==============================] - 1s 131ms/step - loss: 8.9349e-04 - val_loss: 0.0019
Epoch 22/50
6/6 [==============================] - 1s 135ms/step - loss: 9.1962e-04 - val_loss: 0.0020
Epoch 23/50
6/6 [==============================] - 1s 135ms/step - loss: 9.1508e-04 - val_loss: 0.0018
Epoch 24/50
6/6 [==============================] - 1s 136ms/step - loss: 0.0011 - val_loss: 0.0018
Epoch 25/50
6/6 [==============================] - 1s 134ms/step - loss: 9.5685e-04 - val_loss: 0.0017
Epoch 26/50
6/6 [==============================] - 1s 134ms/step - loss: 9.7365e-04 - val_loss: 0.0017
Epoch 27/50
6/6 [==============================] - 1s 135ms/step - loss: 8.6352e-

6/6 [==============================] - 1s 128ms/step - loss: 0.0015 - val_loss: 0.0085
Epoch 42/50
6/6 [==============================] - 1s 127ms/step - loss: 0.0014 - val_loss: 0.0073
Epoch 43/50
6/6 [==============================] - 1s 127ms/step - loss: 0.0017 - val_loss: 0.0081
Epoch 44/50
6/6 [==============================] - 1s 128ms/step - loss: 0.0015 - val_loss: 0.0069
Epoch 45/50
6/6 [==============================] - 1s 126ms/step - loss: 0.0013 - val_loss: 0.0075
Epoch 46/50
6/6 [==============================] - 1s 128ms/step - loss: 0.0014 - val_loss: 0.0065
Epoch 47/50
6/6 [==============================] - 1s 129ms/step - loss: 0.0014 - val_loss: 0.0067
Epoch 48/50
6/6 [==============================] - 1s 135ms/step - loss: 0.0013 - val_loss: 0.0067
Epoch 49/50
6/6 [==============================] - 1s 133ms/step - loss: 0.0013 - val_loss: 0.0063
Epoch 50/50
6/6 [==============================] - 1s 131ms/step - loss: 0.0013 - val_loss: 0.0059
1 day output [[0.83397

6/6 [==============================] - 1s 127ms/step - loss: 0.0053 - val_loss: 0.0202
Epoch 6/50
6/6 [==============================] - 1s 129ms/step - loss: 0.0032 - val_loss: 0.0098
Epoch 7/50
6/6 [==============================] - 1s 123ms/step - loss: 0.0042 - val_loss: 0.0128
Epoch 8/50
6/6 [==============================] - 1s 125ms/step - loss: 0.0022 - val_loss: 0.0180
Epoch 9/50
6/6 [==============================] - 1s 130ms/step - loss: 0.0024 - val_loss: 0.0130
Epoch 10/50
6/6 [==============================] - 1s 139ms/step - loss: 0.0020 - val_loss: 0.0132
Epoch 11/50
6/6 [==============================] - 1s 138ms/step - loss: 0.0018 - val_loss: 0.0157
Epoch 12/50
6/6 [==============================] - 1s 136ms/step - loss: 0.0018 - val_loss: 0.0133
Epoch 13/50
6/6 [==============================] - 1s 147ms/step - loss: 0.0017 - val_loss: 0.0134
Epoch 14/50
6/6 [==============================] - 1s 126ms/step - loss: 0.0018 - val_loss: 0.0144
Epoch 15/50
6/6 [=========

Epoch 28/50
6/6 [==============================] - 1s 141ms/step - loss: 0.0015 - val_loss: 0.0083
Epoch 29/50
6/6 [==============================] - 1s 142ms/step - loss: 0.0015 - val_loss: 0.0083
Epoch 30/50
6/6 [==============================] - 1s 136ms/step - loss: 0.0014 - val_loss: 0.0083
Epoch 31/50
6/6 [==============================] - 1s 137ms/step - loss: 0.0014 - val_loss: 0.0080
Epoch 32/50
6/6 [==============================] - 1s 141ms/step - loss: 0.0015 - val_loss: 0.0081
Epoch 33/50
6/6 [==============================] - 1s 137ms/step - loss: 0.0015 - val_loss: 0.0076
Epoch 34/50
6/6 [==============================] - 1s 135ms/step - loss: 0.0014 - val_loss: 0.0078
Epoch 35/50
6/6 [==============================] - 1s 136ms/step - loss: 0.0015 - val_loss: 0.0074
Epoch 36/50
6/6 [==============================] - 1s 135ms/step - loss: 0.0015 - val_loss: 0.0074
Epoch 37/50
6/6 [==============================] - 1s 136ms/step - loss: 0.0014 - val_loss: 0.0072
Epoch 38/5

1 day output [[0.870852]]
2 day output [[0.8795697]]
3 day output [[0.8849175]]
4 day output [[0.8873364]]
5 day output [[0.8873522]]
6 day output [[0.8854869]]
7 day output [[0.8822088]]
8 day output [[0.87791127]]
9 day output [[0.87290764]]
10 day output [[0.8674367]]
11 day output [[0.8616741]]
12 day output [[0.8557449]]
13 day output [[0.84973454]]
14 day output [[0.8436991]]
15 day output [[0.83767587]]
16 day output [[0.831688]]
17 day output [[0.82575035]]
18 day output [[0.8198727]]
19 day output [[0.8140629]]
20 day output [[0.8083263]]
21 day output [[0.80266964]]
22 day output [[0.7970979]]
23 day output [[0.7916174]]
24 day output [[0.78623414]]
25 day output [[0.7809539]]
26 day output [[0.77578306]]
27 day output [[0.7707265]]
28 day output [[0.76579005]]
29 day output [[0.76097786]]
   gain  loss  prediction
0     1     0           1
1     0     1           1
2     1     0           1
3     1     0           0
4     1     0           0
[0] <class 'numpy.ndarray'>
Epoch